### Config the model

In [46]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')
output=model.invoke("hi")
print(output.content)

Hi there! How can I help you today?


### Config the embedding model

In [47]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")
len(embeddings.embed_query("hi"))

384

## lets take a data embedd it and store in VDB

In [48]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [49]:
loader=DirectoryLoader("../data2",glob="./*.txt",loader_cls=TextLoader)

In [50]:
docs=loader.load()
docs

[Document(metadata={'source': '..\\data2\\usa.txt'}, page_content="🇺🇸 Overview of the U.S. Economy\nThe United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under a capitalist mixed economy, where the private sector dominates, but the government plays a significant regulatory and fiscal role. With a population of over 335 million people and a high level of technological advancement, the U.S. economy thrives on a foundation of consumer spending, innovation, global trade, and financial services. It has a highly diversified structure with strong sectors in technology, healthcare, finance, real estate, defense, and agriculture.\n\nU.S. GDP – Size, Composition, and Global Share\nAs of 2024, the United States’ nominal GDP is estimated to be around $28 trillion USD, accounting for approximately 25% of the global economy. It ranks #1 in the world by nominal GDP, far ahead of China (which ra

In [51]:
docs[0].page_content

"🇺🇸 Overview of the U.S. Economy\nThe United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under a capitalist mixed economy, where the private sector dominates, but the government plays a significant regulatory and fiscal role. With a population of over 335 million people and a high level of technological advancement, the U.S. economy thrives on a foundation of consumer spending, innovation, global trade, and financial services. It has a highly diversified structure with strong sectors in technology, healthcare, finance, real estate, defense, and agriculture.\n\nU.S. GDP – Size, Composition, and Global Share\nAs of 2024, the United States’ nominal GDP is estimated to be around $28 trillion USD, accounting for approximately 25% of the global economy. It ranks #1 in the world by nominal GDP, far ahead of China (which ranks 2nd). The U.S. GDP per capita is also among the highest, hover

In [52]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)

In [53]:
new_docs=text_splitter.split_documents(documents=docs)

In [54]:
new_docs

[Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='🇺🇸 Overview of the U.S. Economy'),
 Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='The United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under a capitalist mixed economy,'),
 Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='It operates under a capitalist mixed economy, where the private sector dominates, but the government plays a significant regulatory and fiscal role. With a population of over 335 million people and a'),
 Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='a population of over 335 million people and a high level of technological advancement, the U.S. economy thrives on a foundation of consumer spending, innovation, global trade, and financial services.'),
 Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='innovation, global trade,

In [55]:
doc_string=[doc.page_content for doc in new_docs]
doc_string

['🇺🇸 Overview of the U.S. Economy',
 'The United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under a capitalist mixed economy,',
 'It operates under a capitalist mixed economy, where the private sector dominates, but the government plays a significant regulatory and fiscal role. With a population of over 335 million people and a',
 'a population of over 335 million people and a high level of technological advancement, the U.S. economy thrives on a foundation of consumer spending, innovation, global trade, and financial services.',
 'innovation, global trade, and financial services. It has a highly diversified structure with strong sectors in technology, healthcare, finance, real estate, defense, and agriculture.',
 'U.S. GDP – Size, Composition, and Global Share',
 'As of 2024, the United States’ nominal GDP is estimated to be around $28 trillion USD, accounting for approximately 

In [56]:
len(doc_string)

55

In [57]:
db=Chroma.from_documents(new_docs,embeddings)

In [58]:
retriever=db.as_retriever(search_kwargs={"k": 3})
retriever.invoke("industrial growth of usa?")

[Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='Looking forward, the U.S. economy is expected to grow at a moderate pace, powered by innovation in AI, green energy, robotics, biotech, and quantum computing. The Biden administration’s Inflation'),
 Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='Looking forward, the U.S. economy is expected to grow at a moderate pace, powered by innovation in AI, green energy, robotics, biotech, and quantum computing. The Biden administration’s Inflation'),
 Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='Looking forward, the U.S. economy is expected to grow at a moderate pace, powered by innovation in AI, green energy, robotics, biotech, and quantum computing. The Biden administration’s Inflation')]

## creation of pydantic class

In [59]:
import operator
from typing import List
from pydantic import BaseModel , Field
from langchain.prompts import PromptTemplate
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph,END

In [60]:
class TopicSelectionParser(BaseModel):
    Topic:str=Field(description="selected topic")
    Reasoning:str=Field(description='Reasoning behind topic selection')

In [61]:
from langchain.output_parsers import PydanticOutputParser

In [62]:
parser=PydanticOutputParser(pydantic_object=TopicSelectionParser)

In [63]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"Topic": {"description": "selected topic", "title": "Topic", "type": "string"}, "Reasoning": {"description": "Reasoning behind topic selection", "title": "Reasoning", "type": "string"}}, "required": ["Topic", "Reasoning"]}\n```'

# How agent state works demo of the agentstate

In [64]:
Agentstate={}

In [65]:
Agentstate["messages"]=[]

In [66]:
Agentstate

{'messages': []}

In [67]:
Agentstate["messages"].append("hi how are you?")

In [68]:
Agentstate["messages"].append("what are you doing?")

In [69]:
Agentstate

{'messages': ['hi how are you?', 'what are you doing?']}

In [70]:
Agentstate["messages"].append("i hope everything fine")

In [71]:
Agentstate

{'messages': ['hi how are you?',
  'what are you doing?',
  'i hope everything fine']}

In [72]:
Agentstate["messages"][-1]

'i hope everything fine'

In [73]:
Agentstate["messages"][0]

'hi how are you?'

### this agentstate class you need to inside the stategraph

In [74]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [78]:
state={"messages":["hi"]}

In [79]:
state

{'messages': ['hi']}

In [84]:
def function_1(state:AgentState):
    
    question=state["messages"][-1]
    
    print("Question",question)
    
    template="""
    Your task is to classify the given user query into one of the following categories: [USA,Not Related]. 
    Only respond with the category name and nothing else.

    User query: {question}
    {format_instructions}
    """
    
    prompt= PromptTemplate(
        template=template,
        input_variable=["question"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    
    
    chain= prompt | model | parser
    
    response = chain.invoke({"question":question})
    
    print("Parsed response:", response)
    
    return {"messages": [response.Topic]}

In [85]:
state={"messages":["what is a today weather?"]}

In [86]:
state={"messages":["what is a GDP of usa??"]}

In [87]:
function_1(state)

Question what is a GDP of usa??
Parsed response: Topic='USA' Reasoning='The query explicitly asks for the GDP of the USA.'


{'messages': ['USA']}

In [88]:
def router(state:AgentState):
    print("-> ROUTER ->")
    
    last_message=state["messages"][-1]
    print("last_message:", last_message)
    
    if "usa" in last_message.lower():
        return "RAG Call"
    else:
        return "LLM Call"

In [89]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [90]:
# RAG Function
def function_2(state:AgentState):
    print("-> RAG Call ->")
    
    question = state["messages"][0]
    
    prompt=PromptTemplate(
        template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:""",
        
        input_variables=['context', 'question']
    )
    
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    result = rag_chain.invoke(question)
    return  {"messages": [result]}

In [91]:
# LLM Function
def function_3(state:AgentState):
    print("-> LLM Call ->")
    question = state["messages"][0]
    
    # Normal LLM call
    complete_query = "Anwer the follow question with you knowledge of the real world. Following is the user question: " + question
    response = model.invoke(complete_query)
    return {"messages": [response.content]}

In [92]:
from langgraph.graph import StateGraph,END

In [93]:
workflow=StateGraph(AgentState)

In [94]:
workflow.add_node("Supervisor",function_1)

In [95]:
workflow.add_node("RAG",function_2)

In [96]:
workflow.add_node("LLM",function_3)

In [97]:
workflow.set_entry_point("Supervisor")

In [98]:
workflow.add_conditional_edges(
    "Supervisor",
    router,
    {
        "RAG Call": "RAG",
        "LLM Call": "LLM",
    }
)

In [99]:
workflow.add_edge("RAG",END)
workflow.add_edge("LLM",END)

In [100]:
app=workflow.compile()

In [101]:
state={"messages":["hi"]}

In [102]:
app.invoke(state)

Question hi
Parsed response: Topic='Not Related' Reasoning="The query 'hi' is a greeting and does not contain any information related to the USA."
-> ROUTER ->
last_message: Not Related
-> LLM Call ->


{'messages': ['hi', 'Not Related', 'Hi there! How can I help you today?']}

In [103]:
state={"messages":["what is a gdp of usa?"]}

In [104]:
app.invoke(state)

Question what is a gdp of usa?
Parsed response: Topic='USA' Reasoning='The query explicitly asks for the GDP of the USA.'
-> ROUTER ->
last_message: USA
-> RAG Call ->


{'messages': ['what is a gdp of usa?',
  'USA',
  'I cannot provide a numerical answer for the US GDP.  The provided text only gives the title "U.S. GDP – Size, Composition, and Global Share," which is not sufficient information.  To find the current GDP, you would need to consult a reliable economic data source.']}

In [105]:
state={"messages":["can you tell me the industrial growth of world's most powerful economy?"]}

In [106]:
state={"messages":["can you tell me the industrial growth of world's poor economy?"]}

In [107]:
result=app.invoke(state)

Question can you tell me the industrial growth of world's poor economy?
Parsed response: Topic='Not Related' Reasoning="The query asks about the industrial growth of the world's poor economies, which is a global economic issue not specifically related to the USA."
-> ROUTER ->
last_message: Not Related
-> LLM Call ->


In [108]:
result["messages"][-1]

'The industrial growth of the world\'s poorest economies is a complex and multifaceted issue, lacking a single, easily quantifiable answer.  There\'s no universally agreed-upon definition of "poor economy," and data collection in these regions is often unreliable or incomplete.  However, we can make some general observations:\n\n* **Uneven Growth:** Industrial growth is highly uneven across the poorest economies. Some countries are experiencing significant growth in specific sectors (e.g., garment manufacturing in Bangladesh,  mining in some African nations), while others remain largely agrarian with minimal industrial development.  This disparity is influenced by factors like access to resources, infrastructure, education, political stability, and global trade policies.\n\n* **Challenges to Industrialization:**  Many of the world\'s poorest countries face significant obstacles to industrialization, including:\n    * **Lack of Infrastructure:** Inadequate transportation, energy, and co